### 词性标注

使用nltk自带的Brown词库进行学习

### 导入所需要的库

In [1]:
import nltk
import sys
from nltk.corpus import brown

### 预处理词库

In [2]:
# 查看数据
brown.tagged_sents()[0:2]

[[('The', 'AT'),
  ('Fulton', 'NP-TL'),
  ('County', 'NN-TL'),
  ('Grand', 'JJ-TL'),
  ('Jury', 'NN-TL'),
  ('said', 'VBD'),
  ('Friday', 'NR'),
  ('an', 'AT'),
  ('investigation', 'NN'),
  ('of', 'IN'),
  ("Atlanta's", 'NP$'),
  ('recent', 'JJ'),
  ('primary', 'NN'),
  ('election', 'NN'),
  ('produced', 'VBD'),
  ('``', '``'),
  ('no', 'AT'),
  ('evidence', 'NN'),
  ("''", "''"),
  ('that', 'CS'),
  ('any', 'DTI'),
  ('irregularities', 'NNS'),
  ('took', 'VBD'),
  ('place', 'NN'),
  ('.', '.')],
 [('The', 'AT'),
  ('jury', 'NN'),
  ('further', 'RBR'),
  ('said', 'VBD'),
  ('in', 'IN'),
  ('term-end', 'NN'),
  ('presentments', 'NNS'),
  ('that', 'CS'),
  ('the', 'AT'),
  ('City', 'NN-TL'),
  ('Executive', 'JJ-TL'),
  ('Committee', 'NN-TL'),
  (',', ','),
  ('which', 'WDT'),
  ('had', 'HVD'),
  ('over-all', 'JJ'),
  ('charge', 'NN'),
  ('of', 'IN'),
  ('the', 'AT'),
  ('election', 'NN'),
  (',', ','),
  ('``', '``'),
  ('deserves', 'VBZ'),
  ('the', 'AT'),
  ('praise', 'NN'),
  ('and', 

由上可知，Brown里面的句子都是自己标注好了的，长这个样子：

    [[('The', 'AT'), ('said', 'VBD'),..., ('that', 'CS')],[],...,[]]

里面的每一个[]表示一个样本

我们需要做的就是：给词加上开始和结束符号，因此我们加的格式要和上面的格式一样

这里我们使用：

(START,START)(END,END)

In [3]:
brown_tags_words = [ ]
for sent in brown.tagged_sents():
    # 先加开头
    brown_tags_words.append( ("START", "START") )
    # 为了省事儿，我们把tag都省略成前两个字母
    brown_tags_words.extend([ (tag[:2], word) for (word, tag) in sent ])
    # 加个结尾
    brown_tags_words.append( ("END", "END") )

In [32]:
# 也就是将上面的里面的[]换成一对('START', 'START')、('END', 'END')
# 处理完：[('START', 'START'),('AT', 'The'), ('VB', 'said'),..., ('CS', 'that'),('END', 'END'),('START', 'START'),...,('END', 'END'),...,('START', 'START'),...,('END', 'END')]
brown_tags_words[0:50]

[('START', 'START'),
 ('AT', 'The'),
 ('NP', 'Fulton'),
 ('NN', 'County'),
 ('JJ', 'Grand'),
 ('NN', 'Jury'),
 ('VB', 'said'),
 ('NR', 'Friday'),
 ('AT', 'an'),
 ('NN', 'investigation'),
 ('IN', 'of'),
 ('NP', "Atlanta's"),
 ('JJ', 'recent'),
 ('NN', 'primary'),
 ('NN', 'election'),
 ('VB', 'produced'),
 ('``', '``'),
 ('AT', 'no'),
 ('NN', 'evidence'),
 ("''", "''"),
 ('CS', 'that'),
 ('DT', 'any'),
 ('NN', 'irregularities'),
 ('VB', 'took'),
 ('NN', 'place'),
 ('.', '.'),
 ('END', 'END'),
 ('START', 'START'),
 ('AT', 'The'),
 ('NN', 'jury'),
 ('RB', 'further'),
 ('VB', 'said'),
 ('IN', 'in'),
 ('NN', 'term-end'),
 ('NN', 'presentments'),
 ('CS', 'that'),
 ('AT', 'the'),
 ('NN', 'City'),
 ('JJ', 'Executive'),
 ('NN', 'Committee'),
 (',', ','),
 ('WD', 'which'),
 ('HV', 'had'),
 ('JJ', 'over-all'),
 ('NN', 'charge'),
 ('IN', 'of'),
 ('AT', 'the'),
 ('NN', 'election'),
 (',', ','),
 ('``', '``')]

### 词统计

我们要把我们所有的词库中拥有的单词与tag之间的关系，做个简单粗暴的统计。

$$P(w_i | t_i) = \frac{count(w_i,t_i)}{count(t_i)}$$

这里NLTK给了我们做统计的工具，也可以自行实现

In [12]:
# 条件频率分布
cfd_tagwords = nltk.ConditionalFreqDist(brown_tags_words)
# 条件概率分布
cpd_tagwords = nltk.ConditionalProbDist(cfd_tagwords, nltk.MLEProbDist)

In [13]:
# 看看统计结果
print("The probability of an adjective (JJ) being 'new' is", cpd_tagwords["JJ"].prob("new"))
print("The probability of a verb (VB) being 'duck' is", cpd_tagwords["VB"].prob("duck"))

The probability of an adjective (JJ) being 'new' is 0.01472344917632025
The probability of a verb (VB) being 'duck' is 6.042713350943527e-05


$$P(t_i|t_{i-1})=\frac{count(t_{i-1},t_i)}{count(t_{i-1})}$$

这个公式跟words没有什么关系。它是属于隐层的马尔可夫链。所以我们先取出所有的tag来。

In [14]:
brown_tags = [tag for (tag, word) in brown_tags_words ]

In [15]:
# count(t{i-1} ti)
# bigram的意思是 前后两个一组，联在一起
cfd_tags= nltk.ConditionalFreqDist(nltk.bigrams(brown_tags))
# P(ti | t{i-1})
cpd_tags = nltk.ConditionalProbDist(cfd_tags, nltk.MLEProbDist)

In [16]:
print("If we have just seen 'DT', the probability of 'NN' is", cpd_tags["DT"].prob("NN"))
print( "If we have just seen 'VB', the probability of 'JJ' is", cpd_tags["VB"].prob("DT"))
print( "If we have just seen 'VB', the probability of 'NN' is", cpd_tags["VB"].prob("NN"))

If we have just seen 'DT', the probability of 'NN' is 0.5057722522030194
If we have just seen 'VB', the probability of 'JJ' is 0.016885067592065053
If we have just seen 'VB', the probability of 'NN' is 0.10970977711020183


现在我们有一句话，"I want to race"， 一套tag，"PP VB TO VB"，那么他们之间的匹配度有多高呢？

其实就是：

$$P(START)*P(PP|START)*P(I|PP)*P(VB|PP)*P(want|VB)*P(TO | VB)\\
* P(to | TO) *P(VB | TO) * P(race | VB)*P(END | VB)$$

In [17]:
prob_tagsequence = cpd_tags["START"].prob("PP") * cpd_tagwords["PP"].prob("I") * \
    cpd_tags["PP"].prob("VB") * cpd_tagwords["VB"].prob("want") * \
    cpd_tags["VB"].prob("TO") * cpd_tagwords["TO"].prob("to") * \
    cpd_tags["TO"].prob("VB") * cpd_tagwords["VB"].prob("race") * \
    cpd_tags["VB"].prob("END")

print( "The probability of the tag sequence 'START PP VB TO VB END' for 'I want to race' is:", prob_tagsequence)

The probability of the tag sequence 'START PP VB TO VB END' for 'I want to race' is: 1.0817766461150474e-14


### Viterbi 的实现

如果我们手上有一句话，怎么知道最符合的tag是哪组呢？

In [18]:
# 首先，我们拿出所有独特的tags（也就是tags的全集）
distinct_tags = set(brown_tags)

In [20]:
# 给定一句话
sentence = ["I", "want", "to", "race" ]
sentlen = len(sentence)

In [21]:
# 开始维特比，从1循环到句子的总长N，记为i；每次都找出以tag X为最终节点，长度为i的tag链
viterbi = [ ]
# 同时，还需要一个回溯器，从1循环到句子的总长N，记为i；把所有的tag X前一个tag记下来
backpointer = [ ]

first_viterbi = { }
first_backpointer = { }
for tag in distinct_tags:
    # 不要为START标签记录任何内容
    if tag == "START": continue
    first_viterbi[ tag ] = cpd_tags["START"].prob(tag) * cpd_tagwords[tag].prob( sentence[0] )
    first_backpointer[ tag ] = "START"

print(first_viterbi)
print(first_backpointer)

{'EX': 0.0, 'HV': 0.0, '*': 0.0, 'BE': 0.0, 'NR': 0.0, 'WP': 0.0, 'RP': 0.0, 'CC': 0.0, 'UH': 0.0, ')-': 0.0, 'FW': 0.0, 'AP': 0.0, '.-': 0.0, 'AB': 0.0, ',-': 0.0, '--': 0.0, 'WR': 0.0, ':-': 0.0, 'RB': 0.0, 'VB': 0.0, 'NP': 1.7319067623793952e-06, 'PN': 0.0, 'QL': 0.0, 'WQ': 0.0, 'TO': 0.0, 'DO': 0.0, "''": 0.0, 'DT': 0.0, 'RN': 0.0, ',': 0.0, 'NI': 3.3324520848931064e-07, ')': 0.0, ':': 0.0, 'OD': 0.0, 'AT': 0.0, 'WD': 0.0, 'CS': 0.0, 'PP': 0.014930900689060006, 'IN': 0.0, 'END': 0.0, '*-': 0.0, 'NN': 1.0580313619573935e-06, '(': 0.0, '.': 0.0, 'JJ': 0.0, '``': 0.0, '(-': 0.0, "'": 0.0, 'MD': 0.0, 'CD': 0.0}
{'EX': 'START', 'HV': 'START', '*': 'START', 'BE': 'START', 'NR': 'START', 'WP': 'START', 'RP': 'START', 'CC': 'START', 'UH': 'START', ')-': 'START', 'FW': 'START', 'AP': 'START', '.-': 'START', 'AB': 'START', ',-': 'START', '--': 'START', 'WR': 'START', ':-': 'START', 'RB': 'START', 'VB': 'START', 'NP': 'START', 'PN': 'START', 'QL': 'START', 'WQ': 'START', 'TO': 'START', 'DO': 

In [22]:
# 以上，是所有的第一个viterbi 和第一个回溯点。存到vitterbi和backpointer两个变量里去
viterbi.append(first_viterbi)
backpointer.append(first_backpointer)

In [23]:
# 看一看目前最好的tag是啥
currbest = max(first_viterbi.keys(), key = lambda tag: first_viterbi[ tag ])
print( "Word", "'" + sentence[0] + "'", "current best two-tag sequence:", first_backpointer[ currbest], currbest)

Word 'I' current best two-tag sequence: START PP


In [24]:
# 开始回溯
for wordindex in range(1, len(sentence)):
    this_viterbi = { }
    this_backpointer = { }
    prev_viterbi = viterbi[-1]
    
    for tag in distinct_tags:
        # START没啥用，可忽略
        if tag == "START": continue
        
        # 如果现在这个tag是X，现在的单词是w，
        # 我们想找前一个tag Y，并且让最好的tag sequence以Y X结尾。
        # 也就是说
        # Y要能最大化：
        # prev_viterbi[ Y ] * P(X | Y) * P( w | X)
        
        best_previous = max(prev_viterbi.keys(),
                            key = lambda prevtag: \
            prev_viterbi[ prevtag ] * cpd_tags[prevtag].prob(tag) * cpd_tagwords[tag].prob(sentence[wordindex]))

        this_viterbi[ tag ] = prev_viterbi[ best_previous] * \
            cpd_tags[ best_previous ].prob(tag) * cpd_tagwords[ tag].prob(sentence[wordindex])
        this_backpointer[ tag ] = best_previous
    
    # 每次找完Y 我们把目前最好的 存一下
    currbest = max(this_viterbi.keys(), key = lambda tag: this_viterbi[ tag ])
    print( "Word", "'" + sentence[ wordindex] + "'", "current best two-tag sequence:", this_backpointer[ currbest], currbest)


    # 完结
    # 全部存下来
    viterbi.append(this_viterbi)
    backpointer.append(this_backpointer)

Word 'want' current best two-tag sequence: PP VB
Word 'to' current best two-tag sequence: VB TO
Word 'race' current best two-tag sequence: IN NN


In [25]:
# 找END，结束
# 找所有以END结尾的tag sequence
prev_viterbi = viterbi[-1]
best_previous = max(prev_viterbi.keys(),
                    key = lambda prevtag: prev_viterbi[ prevtag ] * cpd_tags[prevtag].prob("END"))

prob_tagsequence = prev_viterbi[ best_previous ] * cpd_tags[ best_previous].prob("END")

# 我们这会儿是倒着存的，因为好的在后面
best_tagsequence = [ "END", best_previous ]
# 同理 这里也有倒过来
backpointer.reverse()

In [26]:
# 回溯所有的回溯点，此时，最好的tag就是backpointer里面的current best
current_best_tag = best_previous
for bp in backpointer:
    best_tagsequence.append(bp[current_best_tag])
    current_best_tag = bp[current_best_tag]

In [27]:
# 显示结果
best_tagsequence.reverse()
print( "The sentence was:", end = " ")
for w in sentence: print( w, end = " ")
print("\n")
print( "The best tag sequence is:", end = " ")
for t in best_tagsequence: print (t, end = " ")
print("\n")
print( "The probability of the best tag sequence is:", prob_tagsequence)

The sentence was: I want to race 

The best tag sequence is: START PP VB IN NN END 

The probability of the best tag sequence is: 5.71772824864617e-14
